# The Assignment problem with CVXPY
In the *Assignment* problem we have a real data matrix $C=(c_{ij})_{i,j=1..n}$ and we want to solve the following: 

\begin{align}
\text{minimize } & \sum_{i,j} c_{ij} x_{ij} \\
\text{subject to } & \sum_{j=1}^n x_{ij} = 1 \text{ for all } i=1,\ldots, n \\
& \sum_{i=1}^n x_{ij} = 1 \text{ for all } j=1,\ldots,n \\
& x_{ij} \ge 0 \text{ for all } i,j=1,\ldots, n \\
& x \in {\bf R}^{n \times n}
\end{align}

It is a linear optimization problem. Observe that the objective can also be written as 
$
\mathrm{Tr}(C X^\top). 
$



In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['svg']

## CVXPY Formulation
We implement a function that solves the Assignment problem:
- input: a cost matrix `C` of shape `(n,n)`;
- output: optimal assignment matrix `X`, and the CVXPY `Problem` object.

In [ ]:
# CVXPY formulation
def assignment(C):
    assert C.shape[0] == C.shape[1]
    n = C.shape[0]
    X = cp.Variable((n, n))
    objective = cp.Minimize(cp.trace(C @ X.T))
    # objective = cp.Minimize(cp.sum(cp.multiply(C, X))) # alternative equivalent definition of the objective
    prob = cp.Problem(objective, # objective
                     [cp.sum(X, axis=1) == np.ones(n), # first family of equality constraints
                                                       # (sums along columns of x)
                     cp.sum(X, axis=0) == np.ones(n), # second family of equality constraints
                                                      # (sums along rows of x)
                     X >= 0]) # non-negativity constraints
    # Call the solver
    prob.solve();
    return X, prob

## Example
Suppose we are assigning 4 jobs to 4 persons. Every person-job combination incurs a different cost. The objective is to minimize the total cost. 

| | Job 1 | Job 2 | Job 3 | Job 4 |
| --- | --- | --- | --- | --- |
| Amy | 8 | 4 | 7 | 0 |
| Beth | 5 | 2 | 3 | 0 |
| Carl | 9 | 6 | 7 | 0 |
| David | 9 | 4 | 8 | 0 |

In [ ]:
# Example problem instance
C = np.array([[8, 4, 7, 0],
              [5, 2, 3, 0],
              [9, 6, 7, 0],
              [9, 4, 8, 0]])

In [ ]:
# Let us compute and print the result for the instance at hand
X, prob = assignment(C)
print("The computed optimal value is:", prob.value)
print()
print("An optimal solution is:")
print(X.value)

The computed optimal solution is very close to integral. Let us round it and recompute the value. 

In [ ]:
# Round to an integral matrix
print("The rounded solution is:")
print(np.round(X.value))
print()
print("Its value is:", np.trace(C @ np.round(X.T.value)))

Observe that the rounded matrix is a permutation matrix (in general, *Birkhoff's theorem* shows how to a round a solution without increasing the cost). The corresponding assignment can be illustrated like this: 

| | Job 1 | Job 2 | Job 3 | Job 4 |
| --- | --- | --- | --- | --- |
| Amy | **8** | 4 | 7 | 0 |
| Beth | 5 | 2 | **3** | 0 |
| Carl | 9 | 6 | 7 | **0** |
| David | 9 | **4** | 8 | 0 |

## Discrete Optimal Transport
A special case of the Assignment problem is the _Discrete Optimal Transport_ problem. Here we have two sets of $n$ points each in a metric space and we seek to match each point in the first set with a point in the second set, such that the total distance of the matched pairs is minimized. Let us solve this problem on two randomly generated points sets $S, T \subset {\bf R}^2$. 

In [ ]:
n = 40 # number of points in each set
np.random.seed(1)
S = np.random.random_sample((n, 2)) # sample n points uniformly distributed in [0,1]^2
T = np.random.random_sample((n, 2)) # same for T

In [ ]:
# Plot the dataset
fig, ax = plt.subplots(figsize=(6,6))
ax.scatter(S[:,0], S[:,1])
ax.scatter(T[:,0], T[:,1], c='red');
#plt.show()

In [ ]:
# Form the pairwise distance matrix
C = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        C[i, j] = np.linalg.norm(S[i,:] - T[j,:])

# Solve the assignment problem with cost matrix C
X, prob = assignment(C)

In [ ]:
# Plot the resulting matching
M = np.round(X.value)
fig, ax = plt.subplots(figsize=(6,6))
for i in range(n):
    for j in range(n):
        if M[i, j] > 0: # if i and j are matched...
            ax.plot([S[i,0], T[j,0]], [S[i,1], T[j,1]], c='black') # ...join the matched points
ax.scatter(S[:,0], S[:,1])
ax.scatter(T[:,0], T[:,1], c='red');

**Stop and think.** You may have noticed that no pair of segments crosses. Why do you think that is the case? Can you prove it? 